In [ ]:
pip install -r requirements.txt

In [ ]:
import pandas as pd

In [ ]:
raw_df = pd.read_csv("raw_datasets/FY2023_FileA.csv")

In [ ]:
budget_df = pd.DataFrame()

## Mapping columns to budget table
column_mappings = {
    'treasury_account_symbol':'budget_id',
    'budget_function':'budget_function',
    'budget_subfunction':'budget_subfunction',
    'budget_authority_unobligated_balance_brought_forward':'prev_year_unspent',
    'adjustments_to_unobligated_balance_brought_forward_cpe':'prev_year_adjustments',
    'budget_authority_appropriated_amount':'reserved_amount',
    'borrowing_authority_amount':'allowed_borrow',
    'contract_authority_amount':'pre_setaside_promise',
    'spending_authority_from_offsetting_collections_amount':'collected_spend',
    'total_other_budgetary_resources_amount':'other_sources_amount',
    'total_budgetary_resources':'yearly_spendable',
    'obligations_incurred':'promised_spending',
    'deobligations_or_recoveries_or_refunds_from_prior_year':'prior_year_refunds',
    'unobligated_balance':'unpromised_balance',
    'gross_outlay_amount':'spent_amount',
    'status_of_budgetary_resources_total':'remaining_summary',
    'federal_account_symbol':'agency_id',
    'allocation_transfer_agency_identifier_code':'transfer_agency_id'
}
budget_df = raw_df[list(column_mappings.keys())].copy()
budget_df.rename(columns=column_mappings, inplace=True)

# Creating availablity_id attribute
budget_df['composite_key'] = raw_df['beginning_period_of_availability'].astype(str) + '_' + raw_df['ending_period_of_availability'].astype(str)
unique_keys = budget_df.drop_duplicates(subset=['composite_key'])['composite_key'].unique()
key_to_id = {key: idx + 1 for idx, key in enumerate(unique_keys)}
# Use composite key to create availablity_id key
budget_df['availability_id'] = budget_df['composite_key'].map(key_to_id)
# Drop composite key column
budget_df = budget_df.drop('composite_key', axis=1)
# Creating account_id attribute
budget_df['account_id'] = raw_df.apply(lambda row: f"{row['federal_account_symbol']}-{str(row['sub_account_code']).zfill(2)}", axis=1)

# Show some stats on rows outputted
print(budget_df.count())

# Save cleaned budget data into CSV
budget_df.to_csv('datasets/FT_BUDGET_2023.csv', index=False)

In [ ]:
# Agency Dataframe
agency_df = pd.DataFrame()
agency_df['agency_id'] = raw_df['agency_identifier_code'].copy()
agency_df['agency_name'] = raw_df['agency_identifier_name'].copy()
agency_df = agency_df.drop_duplicates()

# Show some stats on row outputed
print(agency_df.count())

# Save cleaned agency data into CSV
agency_df.to_csv('datasets/DT_AGENCY_2023.csv', index=False)

In [ ]:
# Account Dataframe
account_df = pd.DataFrame()
account_df['account_id'] = raw_df['federal_account_symbol'].copy()
account_df['account_name'] = raw_df['federal_account_name'].copy()
account_df['main_account_num'] = raw_df['main_account_code'].copy()
account_df['sub_account_num'] = raw_df['sub_account_code'].copy()
# append sub_account_num to account_id to ensure it remains unique
account_df['account_id'] = account_df.apply(lambda row: f"{row['account_id']}-{str(row['sub_account_num']).zfill(2)}", axis=1)
account_df = account_df.drop_duplicates()

# Show some stats on row outputed
print(account_df.count())

# Save cleaned agency data into CSV
account_df.to_csv('datasets/DT_ACCOUNT_2023.csv',index=False)

In [ ]:
# Availability Type Dataframe
availability_type_df = pd.DataFrame()
availability_type_df['availability_id'] = budget_df['availability_id'].copy()
availability_type_df['availability_start'] = raw_df['beginning_period_of_availability'].copy()
availability_type_df['availability_end'] = raw_df['ending_period_of_availability'].copy()

# Drop duplicates
availability_type_df = availability_type_df.drop_duplicates()

# Show some stats on row outputed
print(availability_type_df.count())

# Save cleaned agency data into CSV
availability_type_df.to_csv('datasets/DT_AVAILABILITY_TYPE_2023.csv',index=False)